# Este notebook foi executado em um ambiente google colab, faço a recomendação de utilizar o mesmo para execução do projeto

In [ ]:
%pip install huggingface-hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
%%capture
%pip install transformers
%pip install datasets
%pip install evaluate
%pip install torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import os

In [ ]:
dataset = load_dataset("json", data_files={"train": "/content/treino.jsonl", "test": "/content/teste.jsonl"})
dataset = dataset.map(
    lambda examples: {
        "input": examples["prompt"],
        "output": examples["completion"],
    },
    remove_columns=["prompt", "completion"],
)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 100
    })
})

In [ ]:
checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

mapDict = {
    "suporte": 0,
    "venda": 1,
}

def transform_labels(label):
    label = label["output"]
    result = []
    for l in label:
        result.append(mapDict[l])
    return {"label": result}


def tokenize_function(example):
    return tokenizer(example["input"], padding=True, truncation=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.map(transform_labels, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_datasets

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input', 'output', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 100
    })
})

In [ ]:
from transformers import TrainingArguments

output_dir = "./bert-department-classification"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir="./logs.log",
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=200,
    save_total_limit=2,
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",
)

In [ ]:
from transformers import Trainer, AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

os.environ["WANDB_DISABLE"] = "true"
os.environ["WANDB_MODE"] = "offline"

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metric(eval_perd):
  logits, labels = eval_perd
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metric
)

<ipython-input-9-2b6a68398234>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
200,0.000700,0.000395,1.000000


TrainOutput(global_step=315, training_loss=0.05146662784326408, metrics={'train_runtime': 1299.3153, 'train_samples_per_second': 1.924, 'train_steps_per_second': 0.242, 'total_flos': 33403070610000.0, 'train_loss': 0.05146662784326408, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.0003945403441321105,
 'eval_accuracy': 1.0,
 'eval_runtime': 11.3896,
 'eval_samples_per_second': 8.78,
 'eval_steps_per_second': 1.141,
 'epoch': 5.0}

In [ ]:
trainer.save_model()

In [ ]:
trainer.push_to_hub("vitorhugoclz/model-department-classification")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vitorhugoclz/bert-department-classification/commit/95742147678ec0e217fc4188df5ff2ac88c09f81', commit_message='vitorhugoclz/model-department-classification', commit_description='', oid='95742147678ec0e217fc4188df5ff2ac88c09f81', pr_url=None, repo_url=RepoUrl('https://huggingface.co/vitorhugoclz/bert-department-classification', endpoint='https://huggingface.co', repo_type='model', repo_id='vitorhugoclz/bert-department-classification'), pr_revision=None, pr_num=None)